# Train and Host a Keras Sequential Model

This notebook shows how to train and host a Keras Sequential model on SageMaker. The model used for this notebook is a simple deep CNN that was extracted from [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py).

## The dataset
The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) is one of the most popular machine learning datasets. It consists of 60,000 32x32 images belonging to 10 different classes (6,000 images per class). Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://maet3608.github.io/nuts-ml/_images/cifar10.png)

In this tutorial, we will train a deep CNN to recognize these images.

## Set up the environment

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## Download the CIFAR-10 dataset
Downloading the test and training data will take around 5 minutes.

In [ ]:
import utils

utils.cifar10_download()

## Upload the dataset to an S3 bucket

In [ ]:
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/DEMO-cifar10')

`sagemaker_session.upload_data` will upload the CIFAR-10 dataset from this machine to a bucket named **sagemaker-{region}-{*your aws account number*}**, if you don't have this bucket yet, `sagemaker_session` will create it for you.

## Complete source code
Here is the full source code for the model:

In [ ]:
!cat cifar10_cnn.py

Lets take a closer look:

### The model function
This function constitutes the main difference between TensorFlow and Keras models on SageMaker; Keras models have a `keras_model_fn`:

In [ ]:
def keras_model_fn(hyperparameters):
    model = Sequential()

    model.add(InputLayer(input_shape=(HEIGHT, WIDTH, DEPTH), name=PREDICT_INPUTS))
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    _model = tf.keras.Model(inputs=model.input, outputs=model.output)

    opt = RMSprop(lr=hyperparameters['learning_rate'], decay=hyperparameters['decay'])

    _model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    return _model

This function builds and returns a compiled Keras model.

**Note:** The first layer is named `PREDICT_INPUTS`. This serves as a workaround for a known issue where TensorFlow does not recognize the default (or any custom) name for the first layer of Keras models. Furthermore, note that we are wrapping our model in a `tf.keras.Model` before returning it. This serves as a workaround for a known issue where a Sequential model cannot be directly converted into an Estimator. See [here](https://github.com/tensorflow/tensorflow/issues/20552) for more information about the issue.

### Input functions
These functions are similar to those required by any other model using the TensorFlow Estimator API.

In [ ]:
def serving_input_fn(params):
    # Notice that the input placeholder has the same input shape as the Keras model input
    tensor = tf.placeholder(tf.float32, shape=[None, HEIGHT, WIDTH, DEPTH])
    
    # The inputs key PREDICT_INPUTS matches the Keras InputLayer name
    inputs = {PREDICT_INPUTS: tensor}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)


def train_input_fn(training_dir, params):
    return _input(tf.estimator.ModeKeys.TRAIN,
                    batch_size=BATCH_SIZE, data_dir=training_dir)


def eval_input_fn(training_dir, params):
    return _input(tf.estimator.ModeKeys.EVAL,
                    batch_size=BATCH_SIZE, data_dir=training_dir)

The `train_` and `eval_` functions call the `_input` function which returns a properly processed and shuffled (for training) set of images and labels.

## Create a training job using the SageMaker TensorFlow Estimator

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='cifar10_cnn.py',
                       role=role,
                       framework_version='1.9',
                       hyperparameters={'learning_rate': 1e-4, 'decay':1e-6},
                       training_steps=1000, evaluation_steps=100,
                       train_instance_count=1, train_instance_type='ml.c4.xlarge')

estimator.fit(inputs)

**Note**: Keras models have a known issue and cannot be used for distributed (multi-instance) training. Keep `train_instance_count == 1` until the TensorFlow/Keras team support this feature. See [here](https://github.com/tensorflow/tensorflow/issues/14504) for more information about the issue.


## Deploy the trained model

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## Make some predictions
Prediction is not the focus of this notebook, so to verify the endpoint's functionality, we'll simply generate random data in the correct shape and make a prediction.

In [ ]:
# Creating fake prediction data
import numpy as np
data = np.random.randn(1, 32, 32, 3)

predictor.predict(data)

# Cleaning up
To avoid incurring charges to your AWS account for the resources used in this tutorial you need to delete the SageMaker Endpoint:

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)